In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from catboost import CatBoostClassifier 

In [2]:
df_train= pd.read_csv("/kaggle/input/playground-series-s5e11/train.csv")
df_test= pd.read_csv("/kaggle/input/playground-series-s5e11/test.csv")

In [3]:
df_train_processed=df_train.copy()
df_train_processed['education_level'].unique()

array(['High School', "Master's", "Bachelor's", 'PhD', 'Other'],
      dtype=object)

In [4]:
#Mapping for education_level feature
education_map_dic= {'High School':1, "Master's":3, "Bachelor's":2, 'PhD':4, 'Other':0}

df_train_processed['education_level'] = df_train_processed['education_level'].map(education_map_dic)


#Mapping for grade_subgrade
ct=0
sorted_list=sorted(df_train_processed['grade_subgrade'].unique())
grade_map_dic={}

for i in sorted_list:    
    grade_map_dic[i]=ct
    ct+=1    
df_train_processed['grade_subgrade']=df_train_processed['grade_subgrade'].map(grade_map_dic)

df_train_processed['grade_subgrade'].head()


0    12
1    17
2    14
3    25
4    15
Name: grade_subgrade, dtype: int64

In [5]:
print(df_train_processed[['education_level', 'grade_subgrade']].dtypes)

education_level    int64
grade_subgrade     int64
dtype: object


In [6]:
obj_col_list=df_train_processed.select_dtypes(include=['object']).columns.tolist()
print(obj_col_list)
#drop_first true yapmadım önce bu şekilde eğiticem modellerimi
#eğer beklediğim performans olmazsa özellikle logistic regresyon tarafında ve elimde iyi
#model kalmazsa o zaman yeni bir yol düşünürüm veya vaktim kalırsa.

df_train_processed = pd.get_dummies(
    df_train_processed,
    columns=obj_col_list,
    drop_first=False,
    dtype=int,
)
df_train_processed.info(verbose=True)

['gender', 'marital_status', 'employment_status', 'loan_purpose']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 29 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   id                               593994 non-null  int64  
 1   annual_income                    593994 non-null  float64
 2   debt_to_income_ratio             593994 non-null  float64
 3   credit_score                     593994 non-null  int64  
 4   loan_amount                      593994 non-null  float64
 5   interest_rate                    593994 non-null  float64
 6   education_level                  593994 non-null  int64  
 7   grade_subgrade                   593994 non-null  int64  
 8   loan_paid_back                   593994 non-null  float64
 9   gender_Female                    593994 non-null  int64  
 10  gender_Male                      593994 non-null  int64  
 11 

In [7]:
X=df_train_processed.drop(columns=['id','loan_paid_back']).to_numpy()
y=df_train_processed['loan_paid_back'].astype(int).to_numpy()
X_test = df_test.drop('id',axis=1).to_numpy()


In [9]:
# ================================
# 1. Kütüphaneleri Yükle
# ================================
import optuna
import catboost as cb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import json
import joblib
import os
# Uyarıları kapatalım (CatBoost verbose=False olsa bile bazen konuşur)
import warnings
warnings.filterwarnings("ignore")

# Dosya Yolları
DB_PATH = "/kaggle/working/catboost_optuna.db"
MODEL_PATH = "/kaggle/working/best_catboost_model.pkl"
PARAM_PATH = "/kaggle/working/best_catboost_params.json"
LOG_PATH = "/kaggle/working/catboost_tuning_log.txt"
SUBMISSION_PATH = "/kaggle/working/submission.csv"

# ================================
# 2. Optuna Objective Fonksiyonu
# ================================
def objective(trial):
    # --- CatBoost Spesifik Hiperparametreler ---
    params = {
        # Ağaç Sayısı (n_estimators)
        "iterations": trial.suggest_int("iterations", 1000, 3000),
        
        # Öğrenme Hızı
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        
        # Ağaç Derinliği (CatBoost derin ağaçları sever ama GPU'da çok derin yavaşlatır)
        # Genelde 4-10 arası iyidir.
        "depth": trial.suggest_int("depth", 4, 10),
        
        # L2 Regularization (Overfit engellemek için kritik)
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        
        # GPU için çok önemli: Feature'ları kaç parçaya böleceği (Quantization)
        # 128 veya 254 genelde GPU'da en verimlisidir.
        "border_count": trial.suggest_int("border_count", 32, 255),
        
        # Rastgelelik ve gürültü direnci
        "random_strength": trial.suggest_float("random_strength", 1e-9, 10, log=True),
        
        # Bagging (Satır örnekleme) sıcaklığı
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        
        # --- Sabit Ayarlar ---
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "task_type": "GPU",       # <--- GPU BURADA AKTİF EDİLİYOR
        "devices": "0",           # İlk GPU'yu kullan
        "verbose": False,
        "random_seed": 42,
        
        # Erken durdurma (Overfit olmaya başlayınca dur)
        "od_type": "Iter",
        "od_wait": 50
    }

    # --- StratifiedKFold (Senin İstediğin) ---
    # Repeated yerine normal Stratified kullanıyoruz.
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    aucs = []

    # Cross Validation Döngüsü
    # X ve y senin yukarıdaki kodundan (numpy array) geliyor.
    for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        
        # CatBoost Pool (Veri yapısını CatBoost'un sevdiği formata çevirir - Opsiyonel ama hızlıdır)
        train_pool = cb.Pool(X_train, y_train)
        valid_pool = cb.Pool(X_valid, y_valid)

        model = cb.CatBoostClassifier(**params)
        
        model.fit(
            train_pool,
            eval_set=valid_pool,
            early_stopping_rounds=50,
            verbose=False
        )

        # Tahmin (Olasılık 1. sınıf için)
        preds = model.predict_proba(X_valid)[:, 1]
        auc = roc_auc_score(y_valid, preds)
        aucs.append(auc)

    avg_auc = np.mean(aucs)
    
    # Loglama (İlerlemeyi dosyadan takip etmek için)
    with open(LOG_PATH, "a") as f:
        f.write(f"Trial {trial.number}: Mean AUC = {avg_auc:.5f} | Params: {json.dumps(trial.params)}\n")

    return 1 - avg_auc  # Minimize etmek için (1 - AUC) döndürüyoruz

# ================================
# 3. Optuna Çalıştırma (Study)
# ================================
if os.path.exists(DB_PATH):
    print("🔄 Önceki çalışmadan devam ediliyor...")
    study = optuna.load_study(study_name="catboost_opt", storage=f"sqlite:///{DB_PATH}")
else:
    print("🆕 Yeni çalışma başlatılıyor...")
    study = optuna.create_study(study_name="catboost_opt", direction="minimize", storage=f"sqlite:///{DB_PATH}")

# n_trials kaç deneme yapacağını belirler. Zamanına göre artırabilirsin (Örn: 50, 100).
print("🚀 Optuna Başlıyor...")
study.optimize(objective, n_trials=20, show_progress_bar=True)

# ================================
# 4. En İyi Modeli Kaydetme & Submission
# ================================
print(f"\n🏆 En İyi AUC: {1 - study.best_value:.5f}")
print("En İyi Parametreler:", study.best_params)

# En iyi parametreleri alıp GPU ayarlarını tekrar ekleyelim (Optuna sadece değişkenleri tutar)
final_params = study.best_params.copy()
final_params.update({
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "GPU",
    "devices": "0",
    "verbose": 100, # Final eğitiminde logları görelim
    "random_seed": 42
})



🆕 Yeni çalışma başlatılıyor...


[I 2025-11-24 15:32:54,901] A new study created in RDB with name: catboost_opt


🚀 Optuna Başlıyor...


  0%|          | 0/20 [00:00<?, ?it/s]

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


...

[I 2025-11-24 15:58:31,469] Trial 17 finished with value: 0.07848154131277629 and parameters: {'iterations': 2024, 'learning_rate': 0.07918042230794106, 'depth': 5, 'l2_leaf_reg': 5.1907395014454645, 'border_count': 205, 'random_strength': 3.66049658490374e-06, 'bagging_temperature': 0.652098244980335}. Best is trial 15 with value: 0.07686058765818904.


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


[I 2025-11-24 16:00:34,831] Trial 18 finished with value: 0.07789443985294076 and parameters: {'iterations': 2235, 'learning_rate': 0.030672525573734018, 'depth': 6, 'l2_leaf_reg': 6.379468969176361, 'border_count': 237, 'random_strength': 0.9983653232679857, 'bagging_temperature': 0.12957257892492158}. Best is trial 15 with value: 0.07686058765818904.


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


[I 2025-11-24 16:01:40,391] Trial 19 finished with value: 0.07755663492918718 and parameters: {'iterations': 1703, 'learning_rate': 0.14547345864967426, 'depth': 5, 'l2_leaf_reg': 4.791263885019167, 'border_count': 223, 'random_strength': 1.1033748410716952e-05, 'bagging_temperature': 0.11524046111594749}. Best is trial 15 with value: 0.07686058765818904.

🏆 En İyi AUC: 0.92314
En İyi Parametreler: {'iterations': 1878, 'learning_rate': 0.08778309438675642, 'depth': 5, 'l2_leaf_reg': 5.200105684645899, 'border_count': 247, 'random_strength': 7.072569638582633e-07, 'bagging_temperature': 0.1618044742722452}


In [10]:
# ================================
# 1. Kütüphaneleri Yükle
# ================================
import os  # Dosya sistemi için şart
import optuna
import catboost as cb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import json
import joblib
import numpy as np
import pandas as pd

# Uyarıları kapatalım
import warnings
warnings.filterwarnings("ignore")

# Dosya Yolları
DB_PATH = "/kaggle/working/catboost_optuna.db"
MODEL_PATH = "/kaggle/working/best_catboost_model.pkl"
PARAM_PATH = "/kaggle/working/best_catboost_params.json"
LOG_PATH = "/kaggle/working/catboost_tuning_log.txt"
SUBMISSION_PATH = "/kaggle/working/submission.csv"

# ================================
# 2. Optuna Objective Fonksiyonu
# ================================
def objective(trial):
    # --- CatBoost Spesifik Hiperparametreler ---
    params = {
        # Ağaç Sayısı (Default: 1000)
        "iterations": trial.suggest_int("iterations", 1000, 3000),
        
        # Öğrenme Hızı (Default: Otomatiktir ama genelde 0.03 civarı başlar)
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        
        # Ağaç Derinliği (Default: 6)
        "depth": trial.suggest_int("depth", 4, 10),
        
        # L2 Regularization (Default: 3.0)
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1, 10),
        
        # GPU Quantization (Default GPU: 128)
        "border_count": trial.suggest_int("border_count", 32, 255),
        
        # Rastgelelik (Default: 1.0)
        "random_strength": trial.suggest_float("random_strength", 1e-9, 10, log=True),
        
        # Bagging (Default: 1.0)
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 1.0),
        
        # --- Sabit Ayarlar (Değişmez) ---
        "loss_function": "Logloss",
        "eval_metric": "AUC",     # GPU uyarısı verebilir, sorun yok.
        "task_type": "GPU",       # GPU Aktif
        "devices": "0",           # İlk GPU
        "verbose": False,
        "random_seed": 42,
        
        # Erken durdurma
        "od_type": "Iter",
        "od_wait": 50
    }

    # --- StratifiedKFold ---
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    aucs = []

    # Cross Validation Döngüsü
    for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(X, y)):
        
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]
        
        # CatBoost Pool (Hızlandırıcı)
        train_pool = cb.Pool(X_train, y_train)
        valid_pool = cb.Pool(X_valid, y_valid)

        model = cb.CatBoostClassifier(**params)
        
        model.fit(
            train_pool,
            eval_set=valid_pool,
            early_stopping_rounds=50,
            verbose=False
        )

        preds = model.predict_proba(X_valid)[:, 1]
        auc = roc_auc_score(y_valid, preds)
        aucs.append(auc)

    avg_auc = np.mean(aucs)
    
    # Loglama
    with open(LOG_PATH, "a") as f:
        f.write(f"Trial {trial.number}: Mean AUC = {avg_auc:.5f} | Params: {json.dumps(trial.params)}\n")

    return 1 - avg_auc

# ================================
# 3. Optuna Çalıştırma (Study)
# ================================
if os.path.exists(DB_PATH):
    print("🔄 Önceki çalışmadan devam ediliyor...")
    study = optuna.load_study(study_name="catboost_opt", storage=f"sqlite:///{DB_PATH}")
else:
    print("🆕 Yeni çalışma başlatılıyor...")
    study = optuna.create_study(study_name="catboost_opt", direction="minimize", storage=f"sqlite:///{DB_PATH}")

# --- MÜHENDİSLİK HAMLESİ: DEFAULT PARAMETRELERİ ENJEKTE ETMEK ---
# Optuna'ya diyoruz ki: "Maceraya atılmadan önce, şu güvenli limanı (Default) bir dene."
# Böylece en kötü ihtimalle default skorunu yakalarız.
default_params = {
    "iterations": 1000,         # CatBoost Default
    "learning_rate": 0.03,      # Güvenli bir başlangıç hızı (Default otomatiktir ama bu değer yakındır)
    "depth": 6,                 # CatBoost Default
    "l2_leaf_reg": 3.0,         # CatBoost Default
    "border_count": 128,        # GPU Default
    "random_strength": 1.0,     # CatBoost Default
    "bagging_temperature": 1.0  # CatBoost Default
}

print("💉 Default parametreler enjekte ediliyor (Baseline Injection)...")
study.enqueue_trial(default_params)

# Optuna Başlat
print("🚀 Optuna Başlıyor...")
study.optimize(objective, n_trials=20, show_progress_bar=True)

# ================================
# 4. En İyi Modeli Kaydetme & Submission
# ================================
print(f"\n🏆 En İyi AUC: {1 - study.best_value:.5f}")
print("En İyi Parametreler:", study.best_params)

# Final Parametreleri
final_params = study.best_params.copy()
final_params.update({
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "task_type": "GPU",
    "devices": "0",
    "verbose": 100,
    "random_seed": 42
})



🔄 Önceki çalışmadan devam ediliyor...
💉 Default parametreler enjekte ediliyor (Baseline Injection)...
🚀 Optuna Başlıyor...


  0%|          | 0/20 [00:00<?, ?it/s]

Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


[I 2025-11-24 16:04:35,267] Trial 20 finished with value: 0.08232249077838083 and parameters: {'iterations': 1000, 'learning_rate': 0.03, 'depth': 6, 'l2_leaf_reg': 3.0, 'border_count': 128, 'random_strength': 1.0, 'bagging_temperature': 1.0}. Best is trial 15 with value: 0.07686058765818904.


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


...

[I 2025-11-24 16:27:16,132] Trial 37 finished with value: 0.07787134225201964 and parameters: {'iterations': 1531, 'learning_rate': 0.03542249804324472, 'depth': 8, 'l2_leaf_reg': 5.049142419551055, 'border_count': 252, 'random_strength': 0.00022553200545209263, 'bagging_temperature': 0.2340113193134868}. Best is trial 33 with value: 0.07677755907939454.


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


[I 2025-11-24 16:28:09,656] Trial 38 finished with value: 0.07836496181962627 and parameters: {'iterations': 1923, 'learning_rate': 0.21850460515936618, 'depth': 6, 'l2_leaf_reg': 3.331673412374765, 'border_count': 217, 'random_strength': 0.002626282628387676, 'bagging_temperature': 0.39986262938134964}. Best is trial 33 with value: 0.07677755907939454.


Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU
Default metric period is 5 because AUC is/are not implemented for GPU


[I 2025-11-24 16:29:25,602] Trial 39 finished with value: 0.07750179334217866 and parameters: {'iterations': 1248, 'learning_rate': 0.09076253291503618, 'depth': 5, 'l2_leaf_reg': 8.354477722192833, 'border_count': 234, 'random_strength': 2.8200944807523502e-05, 'bagging_temperature': 0.3013722167483481}. Best is trial 33 with value: 0.07677755907939454.

🏆 En İyi AUC: 0.92322
En İyi Parametreler: {'iterations': 1565, 'learning_rate': 0.12352617617825368, 'depth': 4, 'l2_leaf_reg': 4.166838905969808, 'border_count': 247, 'random_strength': 8.677760209459985e-07, 'bagging_temperature': 0.15688835642328666}
